In [48]:
import pandas as pd
import re
import string

In [4]:
trump_data_path = "../DT_data.csv"
hillary_data_path = "../HC_data.csv"

In [5]:
#Load in csvs as dataframes
trump_df = pd.read_csv(trump_data_path)
hillary_df = pd.read_csv(hillary_data_path)

In [ ]:
#Print out columns

# index = 10

# print(trump_df.columns)
# print(trump_df.iloc[index]["CONTENT_LINK"])
# print(trump_df.iloc[index]["CONTENT"])
# print(trump_df.iloc[index]["PROCESSED"])

In [131]:
total_tweets_trump = []
total_tweets_hillary = []

def handle_days_tweets(content):

  tweets, retweets = [], []
  
  content = content.translate(str.maketrans('', '', '[]\'\",'))
  content = [c.strip() for c in re.split("(\s\d+\s+\d+\s)", content) if c != ""]

  i = 0
  while i < len(content):
    if content[i].split()[0].isnumeric():
      i += 1
      continue

    tweet = dict()
    tweet["text"] = content[i]

    randf = content[i+1].split()

    tweet["retweets"] = randf[0]
    tweet["favorites"] = randf[1]

    if (content[i][0] == "@"):
      retweets.append(tweet)
    else:
      tweets.append(tweet)
    i += 2

  return tweets, retweets

def handle_press_release(content):
  content = content.translate(str.maketrans('', '', '[]\\\','))
  content = re.sub(r'^(n\s|\"?\(.*\)\s[a-zA-Z]+\s\d+(th|st|nd|rd)?\s\d{4}\s?.\s?)', "", content)
  return content

def process_content(df):

  df_size = df.shape[0]

  df["TWEETS"] = ["[]"] * df_size
  df["RETWEETS"] = ["[]"] * df_size
  df["NON_TWEETS"] = ["[]"] * df_size

  for c in range(df_size):

    content = df["CONTENT"][c] + " "
    category = df["CONTENT_CATEGORY"][c]

    if (category == "tweet"):
      df["TWEETS"][c], df["RETWEETS"][c] = handle_days_tweets(content)
    else:
      df["NON_TWEETS"][c] = handle_press_release(content)

In [132]:
#Max number tweets = 87
#Max number retweets = 25

process_content(trump_df)
process_content(hillary_df)

/var/folders/_q/723x80zj33lggwq22rns5tf40000gn/T/ipykernel_5888/3731696927.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["TWEETS"][c], df["RETWEETS"][c] = handle_days_tweets(content)
/var/folders/_q/723x80zj33lggwq22rns5tf40000gn/T/ipykernel_5888/3731696927.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["NON_TWEETS"][c] = handle_press_release(content)


In [133]:
#Write tweets to new csvs
trump_df.to_csv(trump_data_path)
hillary_df.to_csv(hillary_data_path)

# Build metadata

In [149]:
df = pd.DataFrame()

In [150]:
# Keep in mind that the tweets are bunched up together :(

# df["DATE"] = list(trump_df["DATE"]) + list(hillary_df["DATE"])
# df["CONTENT_TYPE"] = list(trump_df["CONTENT_TYPE"]) + list(hillary_df["CONTENT_TYPE"])
# df["CONTENT_CATEGORY"] = list(trump_df["CONTENT_CATEGORY"]) + list(hillary_df["CONTENT_CATEGORY"])
# df["CONTENT_LINK"] = list(trump_df["CONTENT_LINK"]) + list(hillary_df["CONTENT_LINK"])
# df["CANDIDATE"] = ["Donald Trump"] * trump_df.shape[0] + ["Hillary Clinton"] * hillary_df.shape[0]
# df["PERSON"] = ["Donald Trump"] * trump_df.shape[0] + ["Hillary Clinton"] * hillary_df.shape[0]

In [151]:
df.head()

,DATE,CONTENT_TYPE,CONTENT_CATEGORY,CONTENT_LINK,CANDIDATE,PERSON
0,2016-01-01,tweet,tweet,https://www.presidency.ucsb.edu/documents/twee...,Donald Trump,Donald Trump
1,2016-01-02,tweet,tweet,https://www.presidency.ucsb.edu/documents/twee...,Donald Trump,Donald Trump
2,2016-01-03,text,press release,https://www.presidency.ucsb.edu/documents/pres...,Donald Trump,Donald Trump
3,2016-01-03,tweet,tweet,https://www.presidency.ucsb.edu/documents/twee...,Donald Trump,Donald Trump
4,2016-01-04,text,press release,https://www.presidency.ucsb.edu/documents/pres...,Donald Trump,Donald Trump


In [141]:
df.to_csv("../metadata.csv")